In [86]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   

# #검색어 입력
# search = 칼부림
# #검색 시작할 페이지 입력
# page = 1   
# #검색 종료할 페이지 입력
# page2 = 2

# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

ValueError: invalid literal for int() with base 10: 'x'

In [3]:
import os
import pandas as pd
df_news = pd.read_csv(os.getcwd()+"/data/칼부림_20230807_04시07분07초.csv")

In [4]:
import os
import glob
import pandas as pd

# 경로에 있는 모든 csv 파일을 리스트로 가져옵니다.
all_csv_files = glob.glob(os.path.join(os.getcwd() + "/data/", "*.csv"))

# 파일의 마지막 수정 시간을 기준으로 정렬합니다.
latest_csv_file = max(all_csv_files, key=os.path.getctime)

# 가장 최근의 csv 파일을 pandas로 읽습니다.
df_news = pd.read_csv(latest_csv_file)

In [5]:
df_news.head(5)

,date,title,link,content
0,2023-08-07 07:40:01,"BTS 슈가 어깨 드러내자…""칼부림 났다"" 신논현역 소동",https://n.news.naver.com/mnews/article/015/000...,"[\n""가스 테러"" ""흉기 난동"" 오인 소동슈가 타투 공개하자 열차서 팬들 ""꺅""비..."
1,2023-08-07 06:24:01,“아들은 피범벅인데 경찰 사과 한마디 없어”…칼부림 오인 중학생 부모 “피꺼솟”,https://n.news.naver.com/mnews/article/016/000...,[\n온라인 커뮤니티 폭로글에 누리꾼들 ‘부글부글’사복경찰 “소속·신분 고지 없고 ...
2,2023-08-07 10:05:06,"칼부림 예고글 울산 지역 초등학교, 휴교…경찰 추적 중",https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[울산=뉴시스] 중구 성안로 울산경찰청 *재판매 및 DB 금지...
3,2023-08-07 11:31:02,"檢, 당근마켓에 '혜화역 칼부림 예고' 30대 구속영장 청구",https://n.news.naver.com/mnews/article/014/000...,[\n\n\n\n\nⓒ News1 DB /사진=뉴스1 [파이낸셜뉴스] 검찰이 인터넷...
4,2023-08-06 09:35:01,“원주역 칼부림하겠다”…예고 글 올린 10대 하루 만에 잡혀,https://n.news.naver.com/mnews/article/023/000...,[\n\t\t\t\t\t\t\t\t\t\t강원 원주역에서 흉기 난동을 벌이겠다는 글...


In [6]:
import openai

# 발급받은 API 키 설정
# OPENAI_API_KEY 

# openai API 키 인증
# openai.api_key = OPENAI_API_KEY

In [7]:
df_news['link+content'] = df_news['link']+' // '+df_news['content']

In [8]:
df_news['link+content'][0]

'https://n.news.naver.com/mnews/article/015/0004876772?sid=102 // [\n"가스 테러" "흉기 난동" 오인 소동슈가 타투 공개하자 열차서 팬들 "꺅"비명에 놀란 시민들 "난동범 있다" 신고\n\n\n\n그룹 방탄소년단(BTS) 멤버 슈가가 라이브 방송에서 타투를 공개하는 모습과 슈가 방송 이후 서울 지하철 9호선에서 벌어진 오인 소동 현장 모습. / 사진=사회관계망서비스(SNS) 캡처최근 잦아진 흉기 난동 사건으로 시민들의 불안감이 커진 가운데, 6일 서울 지하철 9호선 열차에서 발생한 오인 신고 소동은 그룹 방탄소년단(BTS)을 보던 팬들이 소리쳐 놀란 시민들이 급하게 대피하며 벌어진 것으로 파악됐다.경찰과 소방당국에 따르면 이날 오후 8시 36분께 김포공항행 지하철 9호선 급행열차 승객으로부터 "이상한 냄새가 난다", "사람들이 뛰어다니고 넘어지고 있다", "역사 안에 난동범이 있다" 등 신고가 20여건 접수됐다. 이 중에는 생화학 테러 의심 신고도 있었다.신고 후속 조치를 위해 열차가 신논현역에 정차하자 승객들이 급히 뛰쳐나가면서 계단 등에서 넘어져 7명이 찰과상과 타박상을 입은 것으로 전해졌다. 당국은 열차 내부를 확인한 뒤 별다른 특이사항이 없다고 보고 부상자 6명을 병원으로 이송한 뒤 철수했다. 나머지 1명은 귀가했다.당초 승객들이 동요하기 시작한 이유가 정확히 파악되지 않은 상황에서 온라인상에는 신논현역과 열차 내에서 "독가스 테러다", "칼부림이 났다" 등 헛소문이 퍼졌다. 이후 소동이 발생한 정확한 정황은 현장에 있던 목격자들이 온라인에 글을 올리면서 알려졌다.\n\n\n\n사건 당시 슈가의 방송을 보던 팬들이 단체로 환호성을 지르는 모습. / 사진=사회관계망서비스(SNS) 캡처이날 송파구 케이스포돔에서 열린 아이돌 그룹 방탄소년단(BTS) 멤버 슈가(본명 민윤기·30)의 솔로 콘서트를 관람하고 귀가하던 외국인 팬들이 슈가의 사회관계망서비스(SNS) 라이브 방송을 보다가 소리를 지른 게 발단이었

In [40]:
# 결과를 저장할 빈 리스트
answer_list = []
MAX_TOKENS = 4097
# df['content']에서 0부터 10개까지의 텍스트를 반복하여 처리
for text in df_news['link+content'][:20]:  # 여기서 10은 0부터 9번째 텍스트까지 가져오겠다는 의미입니다.
    print(len(text))
    if len(text) >= MAX_TOKENS-500:  # 500은 query와 기타 메시지에 대한 여유 토큰 수입니다.
        continue
    else:
        # 모델 - GPT 3.5 Turbo 선택
        model = "gpt-3.5-turbo"

        # 질문 작성하기
        query = f"이 뉴스 본문에서 언급된 칼부림예고에 대한 지역과 시간, 뉴스 링크, 요약글 , 범죄자에 대한 신상을 자세하게 뽑아줘. 이때, 지역: OO역 , 시간: 00일 00시 ,뉴스 링크 : https://~~, 요약글: ,범죄자 정보: 00세 A씨 이런 형식으로 꼭 맞춰줘. 만약 여러개의 사건이 있으면 하나의 사건만 뽑아줘 그리고 각각의 사건에서 요약글에 지하철역과 같은 장소가 있다면 지역에 넣어줘.범죄자 정보가 없으면 신원 미상으로 해줘.아래는 본문이야 \n {text}" 
        # query = f"Please select the region, time, crime trailer message, and details about the offender for the stabbing report mentioned in this news article. Please use the following format: area: OO station, time: 00:00 on the 00th, crime trailer message : ~~ , offender information: 00-year-old A, etc. If there are multiple cases, select only one case, and in each case, if the crime trailer has a location such as a subway station, put it in the region.Below is the text \n {text}"
        
        # 메시지 설정하기
        messages = [
                {"role": "system", "content": "please say simple"},
                {"role": "user", "content": query}
        ]

        # ChatGPT API 호출하기
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages
        )
        answer = response['choices'][0]['message']['content']

        answer_list.append(answer)

1390
1997
493
616
554
560
725
1349
1817
2388
531
764
397
8758
1132
1471
517
1412
649


In [41]:
answer_list

['지역: 서울 지하철 9호선\n시간: 6일 오후 8시 36분\n뉴스 링크: https://n.news.naver.com/mnews/article/015/0004876772?sid=102\n요약글: 방탄소년단 멤버 슈가의 라이브 방송을 보던 팬들이 서울 지하철 9호선 열차에서 소리를 지르면서 대피하는 장면이 발생했다. 팬들의 소리 속에 가스 테러와 칼부림 소문이 퍼져 혼란이 생겼으며, 이에 따라 일곱 명이 다쳤다.\n범죄자 정보: 신원 미상',
 '지역: 의정부시 금오동 지역\n시간: 6일 오후 10시\n뉴스 링크: https://n.news.naver.com/mnews/article/016/0002179759?sid=102\n요약글: 의정부시에서 흉기 난동범으로 오인받은 16세 중학생 B군이 사복 경찰들의 무리한 진압으로 부상을 입어 파문이 커지고 있다. 경찰은 경찰 신분증을 꺼내려던 순간 피해자가 도망을 가 넘어져 부상을 입었다고 설명했다.\n범죄자 정보: 16세 중학생 B군 (신원 미상)',
 '지역: 울산 중구 성안로\n시간: 7일 오후 7시27분\n뉴스 링크: https://n.news.naver.com/mnews/article/003/0012016794?sid=102\n요약글: 울산의 초등학교에서 흉기 난동을 예고하는 글이 올라와 경찰이 작성자를 추적하고 있다. 경찰은 게시글 작성자의 IP 주소를 추적하고 있으며, 해당 초등학교는 이날 휴교에 들어갔다. 경찰은 예고 글 작성자를 엄중 처벌할 예정이다.\n범죄자 정보: 신원 미상',
 '지역: 혜화역\n시간: 7일\n뉴스 링크: https://n.news.naver.com/mnews/article/014/0005053373?sid=102\n요약글: 서울 종로구 혜화역에서 칼부림을 예고한 30대 남성에 대해 검찰이 구속영장을 청구했습니다. 왕씨는 당근마켓에 혜화역에서 칼부림을 할 것이라는 글을 작성하여 협박한 혐의를 받고 있습니다.\n범죄자 정보: 이름: 왕모씨(31세)',
 '지역: 원주역, 시간: 6

In [42]:
text = "@".join(answer_list)
text = text.replace("\n"," ")

In [43]:
text

'지역: 서울 지하철 9호선 시간: 6일 오후 8시 36분 뉴스 링크: https://n.news.naver.com/mnews/article/015/0004876772?sid=102 요약글: 방탄소년단 멤버 슈가의 라이브 방송을 보던 팬들이 서울 지하철 9호선 열차에서 소리를 지르면서 대피하는 장면이 발생했다. 팬들의 소리 속에 가스 테러와 칼부림 소문이 퍼져 혼란이 생겼으며, 이에 따라 일곱 명이 다쳤다. 범죄자 정보: 신원 미상@지역: 의정부시 금오동 지역 시간: 6일 오후 10시 뉴스 링크: https://n.news.naver.com/mnews/article/016/0002179759?sid=102 요약글: 의정부시에서 흉기 난동범으로 오인받은 16세 중학생 B군이 사복 경찰들의 무리한 진압으로 부상을 입어 파문이 커지고 있다. 경찰은 경찰 신분증을 꺼내려던 순간 피해자가 도망을 가 넘어져 부상을 입었다고 설명했다. 범죄자 정보: 16세 중학생 B군 (신원 미상)@지역: 울산 중구 성안로 시간: 7일 오후 7시27분 뉴스 링크: https://n.news.naver.com/mnews/article/003/0012016794?sid=102 요약글: 울산의 초등학교에서 흉기 난동을 예고하는 글이 올라와 경찰이 작성자를 추적하고 있다. 경찰은 게시글 작성자의 IP 주소를 추적하고 있으며, 해당 초등학교는 이날 휴교에 들어갔다. 경찰은 예고 글 작성자를 엄중 처벌할 예정이다. 범죄자 정보: 신원 미상@지역: 혜화역 시간: 7일 뉴스 링크: https://n.news.naver.com/mnews/article/014/0005053373?sid=102 요약글: 서울 종로구 혜화역에서 칼부림을 예고한 30대 남성에 대해 검찰이 구속영장을 청구했습니다. 왕씨는 당근마켓에 혜화역에서 칼부림을 할 것이라는 글을 작성하여 협박한 혐의를 받고 있습니다. 범죄자 정보: 이름: 왕모씨(31세)@지역: 원주역, 시간: 6일 오후 6시, 뉴스 링크: https://n.news.na

In [44]:
data = []

# 데이터 분할 및 저장
cases = re.split('지역: ', text)[1:]
for case in cases:
    elements = re.split('(시간: |뉴스 링크: |요약글: |범죄자 정보: )', case)
    try:
        location = elements[0].strip()
        time = elements[2].strip()
        link = elements[4].strip()
        message = elements[6].strip()
        criminal_info = elements[8].strip()
        if "@" in criminal_info:
            temp_info = criminal_info.split("@")[0].strip()  # '@' 문자 이후의 텍스트 제거
            criminal_info = temp_info

        data.append([location, time, link, message, criminal_info])
    except IndexError:  # 형식에 맞지 않는 경우
        print(f"Error in parsing: {case}")

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['Location', 'Time', 'Link', 'Text', 'Criminal Info'])
df.head(10)

Error in parsing: 서현역, 시간: 언급되지 않음, 뉴스 링크: https://n.news.naver.com/mnews/article/005/0001629024?sid=110, 요약글: 현재까지 서현역 일대에서 발생한 칼부림 사건에 대한 언급이 있으며, 이에 대한 안전 수칙이나 대책은 아직 정해지지 않았습니다. 범죄자에 대한 세부정보나 신상정보는 언급되지 않았습니다.@


,Location,Time,Link,Text,Criminal Info
0,서울 지하철 9호선,6일 오후 8시 36분,https://n.news.naver.com/mnews/article/015/000...,방탄소년단 멤버 슈가의 라이브 방송을 보던 팬들이 서울 지하철 9호선 열차에서 소리...,신원 미상
1,의정부시 금오동 지역,6일 오후 10시,https://n.news.naver.com/mnews/article/016/000...,의정부시에서 흉기 난동범으로 오인받은 16세 중학생 B군이 사복 경찰들의 무리한 진...,16세 중학생 B군 (신원 미상)
2,울산 중구 성안로,7일 오후 7시27분,https://n.news.naver.com/mnews/article/003/001...,울산의 초등학교에서 흉기 난동을 예고하는 글이 올라와 경찰이 작성자를 추적하고 있다...,신원 미상
3,혜화역,7일,https://n.news.naver.com/mnews/article/014/000...,서울 종로구 혜화역에서 칼부림을 예고한 30대 남성에 대해 검찰이 구속영장을 청구했...,이름: 왕모씨(31세)
4,"원주역,","6일 오후 6시,",https://n.news.naver.com/mnews/article/023/000...,강원 원주역에서 흉기 난동을 벌이겠다는 글을 올린 17세 A군이 경찰에 붙잡혔다. ...,17세 A씨
5,세종남부경찰서,"6일 오후 5시, 6일 오후 10시, 4일 오후 9시",https://n.news.naver.com/mnews/article/001/001...,세종남부경찰서에서는 중학생 A군과 B군이 각각 전날 오후 5시와 오후 10시께 페이...,세종남부경찰서에서 잡힌 10대들의 신상은 미상이다.
6,부산 서면,6일 10시 40분,https://n.news.naver.com/mnews/article/025/000...,부산 서면에서 흉기 난동을 예고하는 글을 자신의 SNS에 올린 현역 군인 A 일병이...,20세 A 씨
7,"신림동,","6월 22일부터 7월 3일,",https://n.news.naver.com/mnews/article/079/000...,지난달 22일부터 이달 3일까지 신림동에서 발생한 칼부림 사건으로 인해 호신용품 거...,신원 미상
8,분당 서현역,3일 00시,https://n.news.naver.com/mnews/article/023/000...,지난 3일 분당 서현역에서 발생한 '묻지 마 칼부림' 사건 피의자 최모(22)씨와 ...,00세 A씨 (정보 없음)
9,원주역,5일 오전 2시,https://n.news.naver.com/mnews/article/001/001...,강원경찰청 사이버범죄수사대는 사회관계망서비스(SNS)에 살인 예고 글을 올린 혐의(...,17세 A씨


In [15]:
data = []
import re

# 데이터 분할 및 저장
cases = re.split('지역: ', text)[1:]
for case in cases:
    elements = re.split('(시간: |뉴스 링크: |요약글: |범죄자 정보: )', case)
    location = elements[0].strip()
    time = elements[2].strip()
    link = elements[4].strip()
    message = elements[6].strip()
    try :
        criminal_info = elements[8].strip()
        if "@" in criminal_info:
            temp_info = criminal_info.split("@")[0].strip()  # '@' 문자 이후의 텍스트 제거
            criminal_info = temp_info
    except:
        print(elements)

    data.append([location, time, link, message, criminal_info])

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['Location', 'Time', 'Link', 'Text', 'Criminal Info'])

df

['의정부시 금오동 지하철역 ', '시간: ', '6일 오후 10시 ', '뉴스 링크: ', 'https://n.news.naver.com/mnews/article/016/0002179759?sid=102 ', '요약글: ', '경기도 의정부시 금오동에서 사복 경찰들이 16세 중학생 B군을 오인하여 무리한 진압을 가해 부상을 입힌 사건이 발생했다. 아이들의 경찰에 대한 신고로 출동한 사복 경찰들은 B군과 마주쳤고, B군은 경찰들의 폭력적인 대응에 겁을 먹고 반대 방향으로 도망가다가 경찰에 압박당해 강제로 연행되었다고 주장하고 있다. 사건으로 인해 B군은 온 몸에 찰과상이 생긴 상태이다. 범죄자에 대한 신상: 범죄자는 사복 경찰들이지만, 자세한 신상은 없으니 신원 미상으로 처리한다.@']
['울산, ', '시간: ', '7일 오후 7시27분, ', '뉴스 링크: ', 'https://n.news.naver.com/mnews/article/003/0012016794?sid=102, ', '요약글: ', "울산의 초등학교에서 흉기 난동을 예고하는 글이 작성되어 경찰이 작성자를 추적하고 있습니다. 해당 게시글은 '내일 울산 북구 A초등학교에서 칼부림 예정'이라는 내용을 포함하고 있었습니다. 경찰은 작성자의 IP 주소를 추적하고 있으며, 해당 학교는 현재 휴교 상태입니다. 범죄자 정보는 미상입니다.@"]
['서현역, ', '시간: ', '불명 ', '뉴스 링크: ', 'https://n.news.naver.com/mnews/article/005/0001629024?sid=110 ', '요약글: ', '뉴스 본문에서 언급된 서현역 일대 묻지마 칼부림 사건은 아무런 안전수칙이 없는 정상적인 상황에서 일어난 범죄사건입니다. 이 사건은 40건 이상의 살인 예고와 일련의 사회적 공포를 유발하였습니다. 범인의 정보는 최모씨로 추정되며, 그 외에는 많은 자세한 정보가 없습니다.  ']
['신림동 ', '뉴스 링크: ', 'https://n.news.naver.com/mnews/ar

,Location,Time,Link,Text,Criminal Info
0,서울 지하철 9호선,6일 오후 8시 36분,https://n.news.naver.com/mnews/article/015/000...,서울 지하철 9호선에서 발생한 오인 신고 소동은 방탄소년단(BTS)을 보던 팬들이 ...,신원 미상
1,의정부시 금오동 지하철역,6일 오후 10시,https://n.news.naver.com/mnews/article/016/000...,경기도 의정부시 금오동에서 사복 경찰들이 16세 중학생 B군을 오인하여 무리한 진압...,신원 미상
2,"울산,","7일 오후 7시27분,",https://n.news.naver.com/mnews/article/003/001...,울산의 초등학교에서 흉기 난동을 예고하는 글이 작성되어 경찰이 작성자를 추적하고 있...,신원 미상
3,"혜화역,",5일 오후 3시에서 12시 사이,https://n.news.naver.com/mnews/article/014/000...,검찰이 당근마켓에 혜화역에서 칼부림을 예고한 30대 남성에 대해 구속영장을 청구했습...,31세 A씨 (확인되지 않음)
4,"원주역,","5월 5일 오전 2시 18분,",https://n.news.naver.com/mnews/article/023/000...,강원 원주역에서 흉기 난동을 벌이겠다는 글을 올린 17세 A군이 경찰에 붙잡혔다. ...,17세 A군 (추가 정보 없음)
5,세종,"6일 오후 5시, 10시",https://n.news.naver.com/mnews/article/001/001...,세종에서 온라인에 살인 예고 글을 올린 10대들이 잇따라 경찰에 붙잡혔습니다. 중학...,신원 미상
6,부산 서면역,8월 6일 00시,https://n.news.naver.com/mnews/article/025/000...,부산 서면에서 흉기 난동을 예고하는 글을 자신의 SNS에 올린 현역 군인이 경찰에 ...,20세 A씨
7,"신림동,","6월 22일부터 이달 3일까지,",https://n.news.naver.com/mnews/article/079/000...,서울 신림동에서 칼부림 사건이 발생한 다음 날인 지난달 22일부터 이달 3일까지 1...,신원 미상
8,"서현역,",불명,https://n.news.naver.com/mnews/article/005/000...,뉴스 본문에서 언급된 서현역 일대 묻지마 칼부림 사건은 아무런 안전수칙이 없는 정상...,신원 미상
9,신림동,https://n.news.naver.com/mnews/article/005/000...,"뉴스 본문에서 언급된 신림동 칼부림 사건의 범인은 조선씨로 알려져 있으며, 범인은 ...","서현역 일대 묻지마 칼부림 사건의 범인은 최씨로 추정되며, 최근 3년 동안 정신질환...",신원 미상


In [58]:
df

,Location,Time,Link,Criminal Info
0,원주역,5일 오전 2시,https://n.news.naver.com/mnews/article/001/001...,17세 A씨
1,원주역,5일 오후 11시,https://n.news.naver.com/mnews/article/023/000...,A(17)세
2,"원주역,",5일 밤 11시,https://n.news.naver.com/mnews/article/003/001...,17세 A씨
3,서현역,6일 오전 2시,https://n.news.naver.com/mnews/article/081/000...,신원 미상 (살인 등 혐의로 구속됨)
4,경기 고양시 덕양구 상가,5일 자정,https://n.news.naver.com/mnews/article/055/000...,10대 A군
5,서현역,4일,https://n.news.naver.com/mnews/article/016/000...,신원 미상
6,"혜화역,",5일 오후 3시에서 12시 사이,https://n.news.naver.com/mnews/article/437/000...,30세 A씨
7,"서울역,",4일 16시,https://n.news.naver.com/mnews/article/009/000...,20대 남성 (해당 본문에서는 다른 사건에 대한 이야기도 나왔으므로 해당 사건을 ...
8,구미역,5일 오전 3시 30분,https://n.news.naver.com/mnews/article/088/000...,17세 A씨
9,"청주역,","4일 오후 7시,",https://n.news.naver.com/mnews/article/421/000...,30세 A씨


In [83]:
df = df.replace(",","",regex=True)

In [84]:
df

,Location,Time,Link,Text,Criminal Info
0,원주역,5일 오전 2시,https://n.news.naver.com/mnews/article/001/001...,강원경찰청 사이버범죄수사대는 사회관계망서비스(SNS)에 살인 예고 글을 올린 혐의(...,이름 미상 17세
1,원주역,5일 오전 2시 18분,https://n.news.naver.com/mnews/article/023/000...,강원 원주역에서 흉기 난동을 벌이겠다는 글을 올린 10대가 경찰에 붙잡혔다. 강원경...,17세 A씨
2,원주역,5일 밤 11시,https://n.news.naver.com/mnews/article/003/001...,강원 원주역에서 '묻지마 흉기 난동'을 벌이겠다고 예고한 10대가 경찰에 붙잡혔다....,17세 A씨
3,서현역,3일 오후 5시 59분,https://n.news.naver.com/mnews/article/081/000...,경기도 성남시 서현역에서 발생한 '묻지마 칼부림' 난동 사건의 피해자 중 1명이 6...,최모(22세)
4,경기 고양시 덕양구 상가,5일 자정,https://n.news.naver.com/mnews/article/055/000...,경기 고양시 덕양구의 한 상가에서 칼부림을 예고하는 글을 올린 10대가 경찰에 붙잡...,10대 A씨 (신원 미상)
5,신림동 지하철역: 서현역,7월 22일 - 8월 3일,https://n.news.naver.com/mnews/article/016/000...,신림동과 서현역에서 발생한 칼부림 사건 이후 호신용품 수요 급증. 6일간 호신용품 ...,신원 미상
6,혜화역,5일 오후 3시에서 12시 사이,https://n.news.naver.com/mnews/article/437/000...,서울 혜화역에서 흉기 난동을 벌이겠다는 글을 올린 30대 A씨가 경찰에 붙잡혔습니다...,30세 A씨
7,서울역,4일 16시 (요약글에 포함),https://n.news.naver.com/mnews/article/009/000...,전국에서 살인을 예고하는 온라인 게시글이 잇따라 나오고 있으며 경찰이 대대적인 수사...,20대 남성 특정 학교를 거론한 미성년자.
8,구미역,5일 오전 3시 30분,https://n.news.naver.com/mnews/article/088/000...,경북 구미경찰서는 구미역 흉기 난동 예고글을 게시한 17세 A씨를 검거했습니다. A...,17세 A씨
9,청주역,5일 오후 7시,https://n.news.naver.com/mnews/article/421/000...,30대 A씨가 칼부림 예고 목록을 만들어 유포하던 중 경찰에 붙잡혔습니다. A씨는 ...,30세 A씨


In [85]:
import datetime
now = datetime.datetime.now() 
df.to_csv('{}_{}.csv'.format('final_data',now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

In [91]:
import datetime
now = datetime.datetime.now() 
answer_list = pd.DataFrame(answer_list, columns = ['gpt_answer'])
answer_list.to_csv('data/{}_{}.csv'.format('gpt_answer_list',now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

In [39]:
import schedule
import time

# step2.실행할 함수 선언
def message():
    print("스케쥴 실행중...")

# step3.실행 주기 설정
schedule.every(1).seconds.do(message)

# step4.스캐쥴 시작
while True:
    schedule.run_pending()
    time.sleep(1)

스케쥴 실행중...
스케쥴 실행중...
스케쥴 실행중...
스케쥴 실행중...
스케쥴 실행중...
스케쥴 실행중...
스케쥴 실행중...
스케쥴 실행중...
스케쥴 실행중...


KeyboardInterrupt: 